In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Config

In [86]:
test_split = 0.05
validation_split = 0.05
learning_rate = 1e-3
input_size = 300
hidden_size = 225
bottleneck_size = 150
epochs = 5
batch_size = 128
use_hidden_layer = True
experiment_name = "baseline_autoencoder_v3.2"
dataset = "data/embeddings/base/clipped.glove.6B.300d.txt"

In [87]:
gpu = torch.cuda.is_available()
device = torch.device("cuda" if gpu else "cpu")

# Load & Prepare Embeddings for Training

In [71]:
words = []
vectors = []
with open(dataset, "r", encoding='utf8') as fp:
    for line in fp:
        line = line.split()
        word = line[0]
        vector = np.asarray(line[1:], 'float32')
        words.append(word)
        vectors.append(vector)
vectors = torch.from_numpy(np.asarray(vectors))

In [72]:
test_split = int(test_split * len(words))
validation_split = int(validation_split* len(words))

In [80]:
print(len(words), vectors.shape)
print(test_split, validation_split)

400000 torch.Size([400000, 300])
20000 20000


In [81]:
train_vectors, test_vectors, train_words, test_words = train_test_split(
    vectors, words, test_size=test_split, random_state=seed
)

In [82]:
train_vectors, validation_vectors, train_words, validation_words = train_test_split(
    train_vectors, train_words, test_size=validation_split, random_state=seed
)

In [83]:
train_vectors.shape, validation_vectors.shape, test_vectors.shape

(torch.Size([360000, 300]), torch.Size([20000, 300]), torch.Size([20000, 300]))

In [88]:
# Note: We don't actually use these words since the model doesn't care about them.
# We just compute them in case we want to check some particular word or something.
len(train_words), len(validation_words), len(test_words)

(360000, 20000, 20000)

In [89]:
train_dataloader = torch.utils.data.DataLoader(
    train_vectors, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu
)
validation_dataloader = torch.utils.data.DataLoader(
    validation_vectors, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu
)
test_dataloader = torch.utils.data.DataLoader(
    test_vectors, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu
)

# Model Time

In [90]:
class AutoEncoderWithoutHiddenLayer(nn.Module):
    def __init__(self, input_size, bottleneck_size):
        super().__init__()
        self.encoder = nn.Linear(in_features=input_size, out_features=bottleneck_size)
        self.encoder_activation = nn.Tanh()
        
        self.decoder = nn.Linear(in_features=bottleneck_size, out_features=input_size)
        self.decoder_activation = nn.Tanh()
        
        self.decoder.weight = nn.Parameter(self.encoder.weight.transpose(0,1))
    
    def forward(self, features):
        latent_representation = self.encoder_activation(self.encoder(features))
        reconstructed_input = self.decoder_activation(self.decoder(latent_representation))
        return reconstructed_input
    
    def encode(self, features):
        return self.encoder_activation(self.encoder(features))

In [91]:
class AutoEncoderWithHiddenLayer(nn.Module):
    def __init__(self, input_size, hidden_size, bottleneck_size):
        super().__init__()
        self.encoder_input = nn.Linear(in_features=input_size, out_features=hidden_size)
        self.encoder_input_activation = nn.ReLU(True)
        self.encoder_hidden = nn.Linear(in_features=hidden_size, out_features=bottleneck_size)
        self.encoder_hidden_activation = nn.Tanh()
        
        self.decoder_hidden = nn.Linear(in_features=bottleneck_size, out_features=hidden_size)
        self.decoder_hidden_activation = nn.ReLU(True)
        self.decoder_output = nn.Linear(in_features=hidden_size, out_features=input_size)
        self.decoder_output_activation = nn.Tanh()
        
        self.decoder_hidden.weight = nn.Parameter(self.encoder_hidden.weight.transpose(0,1))
        self.decoder_output.weight = nn.Parameter(self.encoder_input.weight.transpose(0,1))
        
        self.encoder = [
            self.encoder_input, self.encoder_input_activation, self.encoder_hidden, self.encoder_hidden_activation
        ]
        self.decoder = [
            self.decoder_hidden, self.decoder_hidden_activation, self.decoder_output, self.decoder_output_activation
        ]
    
    def forward(self, features):
        # Encoder
        for layer in self.encoder:
            features = layer(features)
        # Decoder
        for layer in self.decoder:
            features = layer(features)
        return features
    
    def encode(self, features):
        for layer in self.encoder:
            features = layer(features)
        return features

In [92]:
if use_hidden_layer:
    model = AutoEncoderWithHiddenLayer(input_size, hidden_size, bottleneck_size).to(device)
else:
    model = AutoEncoderWithoutHiddenLayer(input_size, bottleneck_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [93]:
model

AutoEncoderWithHiddenLayer(
  (encoder_input): Linear(in_features=300, out_features=225, bias=True)
  (encoder_input_activation): ReLU(inplace=True)
  (encoder_hidden): Linear(in_features=225, out_features=150, bias=True)
  (encoder_hidden_activation): Tanh()
  (decoder_hidden): Linear(in_features=150, out_features=225, bias=True)
  (decoder_hidden_activation): ReLU(inplace=True)
  (decoder_output): Linear(in_features=225, out_features=300, bias=True)
  (decoder_output_activation): Tanh()
)

In [94]:
for epoch in range(epochs):
    train_loss = 0
    validation_loss = 0
    
    # Training Loop
    for iteration, batch in enumerate(tqdm(train_dataloader)):
        # Move batch to device
        batch = batch.to(device)
        
        # Reset gradients back to zero for this iteration
        optimizer.zero_grad()
        
        # Run our model & get outputs
        outputs = model(batch)
        
        # Calculate reconstruction loss
        batch_loss = criterion(outputs, batch)
        
        # Backprop
        batch_loss.backward()
        
        # Update our optimizer parameters
        optimizer.step()
        
        # Add the batch's loss to the total loss for the epoch
        train_loss += batch_loss.item()
        
    # Validation Loop
    with torch.no_grad():
        for iteration, batch in enumerate(tqdm(validation_dataloader)):
            # Move batch to device
            batch = batch.to(device)

            # Run our model & get outputs
            outputs = model(batch)

            # Calculate reconstruction loss
            batch_loss = criterion(outputs, batch)

            # Add the batch's loss to the total loss for the epoch
            validation_loss += batch_loss.item()
    
    # Compute the average losses for this epoch
    train_loss = train_loss / len(train_dataloader)
    validation_loss = validation_loss / len(validation_dataloader)
    
    # Print Metrics
    print(
        f"Epoch: {epoch+1}/{epochs}, Train Reconstruction Loss = {train_loss}, \
        Validation Reconstruction Loss = {validation_loss}"
    )

  0%|                                                                                         | 0/2813 [00:00<?, ?it/s]

Epoch: 1/5, Train Reconstruction Loss = 0.04949438800638654,         Validation Reconstruction Loss = 0.045302523905114765


  0%|                                                                                         | 0/2813 [00:00<?, ?it/s]

Epoch: 2/5, Train Reconstruction Loss = 0.045042383778870765,         Validation Reconstruction Loss = 0.04549318632692288


  0%|                                                                                         | 0/2813 [00:00<?, ?it/s]

Epoch: 3/5, Train Reconstruction Loss = 0.04553679481461258,         Validation Reconstruction Loss = 0.04667897176021223


  0%|                                                                                         | 0/2813 [00:00<?, ?it/s]

Epoch: 4/5, Train Reconstruction Loss = 0.045967318583776465,         Validation Reconstruction Loss = 0.047221679928576114


100%|███████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 134.85it/s]

Epoch: 5/5, Train Reconstruction Loss = 0.04639475092350102,         Validation Reconstruction Loss = 0.04671787914292068


# Test Model

In [95]:
model.eval()

AutoEncoderWithHiddenLayer(
  (encoder_input): Linear(in_features=300, out_features=225, bias=True)
  (encoder_input_activation): ReLU(inplace=True)
  (encoder_hidden): Linear(in_features=225, out_features=150, bias=True)
  (encoder_hidden_activation): Tanh()
  (decoder_hidden): Linear(in_features=150, out_features=225, bias=True)
  (decoder_hidden_activation): ReLU(inplace=True)
  (decoder_output): Linear(in_features=225, out_features=300, bias=True)
  (decoder_output_activation): Tanh()
)

In [96]:
reconstruction_loss = 0

# Testing Loop
with torch.no_grad():
    for iteration, batch in enumerate(tqdm(test_dataloader)):
        # Move batch to device
        batch = batch.to(device)

        # Run our model & get outputs
        outputs = model(batch)

        # Calculate reconstruction loss
        batch_loss = criterion(outputs, batch)

        # Add the batch's loss to the total loss for the epoch
        reconstruction_loss += batch_loss.item()

# Compute the average losses for this epoch
reconstruction_loss = reconstruction_loss / len(test_dataloader)

# Print Metrics
print(
    f"Test Reconstruction Loss = {reconstruction_loss}"
)

100%|███████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 128.24it/s]

Test Reconstruction Loss = 0.046632874353676086


# Generate Latent Embeddings

In [97]:
latent_vectors = {}
with torch.no_grad():
    for i, (word, vector) in enumerate(tqdm(zip(words, vectors))):
        latent_vectors[word] = model.encode(vector)

400000it [00:33, 11812.06it/s]


In [98]:
len(latent_vectors)

400000

In [99]:
latent_vectors['the']

tensor([-6.4498e-02, -7.6930e-01,  2.5137e-01, -4.8166e-01,  7.5420e-02,
        -3.8943e-02,  3.8439e-01, -3.0558e-02,  7.7318e-01,  1.1349e-01,
         3.4453e-01,  6.5616e-01,  3.6993e-02, -3.8913e-01, -5.0751e-01,
        -4.1768e-01,  8.7623e-03, -1.4096e-01,  5.8439e-01,  9.4979e-02,
        -4.4509e-01,  4.4437e-01, -8.9866e-03, -3.2963e-01,  4.4794e-01,
        -9.5649e-02,  2.6015e-01, -5.5979e-01, -7.2274e-01,  5.3344e-01,
        -2.8207e-01, -2.1124e-01, -3.8037e-01,  5.0595e-01, -2.3402e-01,
         3.9700e-01,  3.3881e-01,  3.4063e-01,  5.4901e-01,  6.0589e-03,
        -3.2924e-01, -2.1518e-01,  2.8385e-01,  2.6296e-02,  5.4774e-01,
        -5.2363e-01,  3.8973e-01,  1.9321e-01, -2.4227e-01, -5.2056e-01,
        -4.4525e-01, -2.8845e-01, -3.8041e-01,  8.2646e-02,  3.3862e-01,
         3.4322e-01, -4.4412e-01,  3.1770e-02,  3.4585e-01,  1.8332e-01,
         7.8772e-02,  1.6246e-01,  1.0525e-01,  5.5264e-01,  2.6130e-01,
        -5.1454e-01,  1.4568e-02,  6.4210e-01, -5.4

# Save Model & Embeddings

In [100]:
torch.save(model.state_dict(), f"models/{experiment_name}.pt")

In [101]:
# Need to convert the latent embeddings into the glove format
# word dim1 dim2 dim3 dim4 ... dimX
lines = []
for i, (word, vector) in tqdm(enumerate(latent_vectors.items())):
    line = [word] + [str(x) for x in vector.tolist()]
    lines.append(' '.join(line))

400000it [00:42, 9477.42it/s]


In [102]:
with open(f"data/embeddings/trained/{experiment_name}.glove.6B.300d.txt", "w", encoding="utf-8") as fp:
    fp.write("\n".join(lines))